In [6]:
!pip3 install pymongo

In [7]:
import pymongo
from pymongo import MongoClient
import datetime
import pandas as pd

In [3]:
client=MongoClient("20.51.211.46", 27000, username="admin", password="abc123")

In [8]:
client.server_info()


{'version': '7.0.0',
 'gitVersion': '37d84072b5c5b9fd723db5fa133fb202ad2317f1',
 'modules': [],
 'allocator': 'tcmalloc',
 'javascriptEngine': 'mozjs',
 'sysInfo': 'deprecated',
 'versionArray': [7, 0, 0, 0],
 'openssl': {'running': 'OpenSSL 1.1.1f  31 Mar 2020',
  'compiled': 'OpenSSL 1.1.1f  31 Mar 2020'},
 'buildEnvironment': {'distmod': 'ubuntu2004',
  'distarch': 'x86_64',
  'cc': '/opt/mongodbtoolchain/v4/bin/gcc: gcc (GCC) 11.3.0',
  'ccflags': '-Werror -include mongo/platform/basic.h -ffp-contract=off -fasynchronous-unwind-tables -g2 -Wall -Wsign-compare -Wno-unknown-pragmas -Winvalid-pch -gdwarf-5 -fno-omit-frame-pointer -fno-strict-aliasing -O2 -march=sandybridge -mtune=generic -mprefer-vector-width=128 -Wno-unused-local-typedefs -Wno-unused-function -Wno-deprecated-declarations -Wno-unused-const-variable -Wno-unused-but-set-variable -Wno-missing-braces -fstack-protector-strong -gdwarf64 -Wa,--nocompress-debug-sections -fno-builtin-memcmp -Wimplicit-fallthrough=5',
  'cxx': '

gmr database was created on MongoDB

The Google Mobility data was imported into a collection named mobility in the assignment database using the code " mongoimport --db gmr --collection employees --type csv --file employees.csv --host "20.51.211.46:27000" --headerline"

In [9]:
client.list_database_names()

['admin', 'config', 'gmr', 'local']

In [10]:
# access the data collection
coviddb=client['gmr']
coviddb

Database(MongoClient(host=['20.51.211.46:27000'], document_class=dict, tz_aware=False, connect=True), 'gmr')

In [11]:
# access collection containing all the data
Mobility=coviddb['Mobility']
Mobility

Collection(Database(MongoClient(host=['20.51.211.46:27000'], document_class=dict, tz_aware=False, connect=True), 'gmr'), 'Mobility')

In [14]:
# Create collection that contains data for only Australia

Mobility = coviddb["Mobility"]
Australia = coviddb["Australia"]

filter_criteria = {"country_region": "Australia"}

aus_data = Mobility.find(filter_criteria)

Australia.insert_many(aus_data)

In [17]:
## Remove unwanted fields from Australia collection

fields_to_remove = ['country_region_code', 'metro_area', 'iso_3166_2_code', 'census_fips_code', 'place_id'] #list of field names to remove

for field in fields_to_remove:
    Australia.update_many({}, {"$unset": {field: 1}}) # Looping through the documents in the collection to remove the specified fields



In [18]:
# Rename some of the fields

field_rename_mapping = {
    "country_region": "country",
    "sub_region_1": "state",
    "sub_region_2": "region",
    "retail_and_recreation_percent_change_from_baseline": "retail_and_recreation",
    "grocery_and_pharmacy_percent_change_from_baseline": "grocery_and_pharmacy",
    "parks_percent_change_from_baseline": "parks",
    "transit_stations_percent_change_from_baseline": "transit_stations",
    "workplaces_percent_change_from_baseline":"workplaces",
    "residential_percent_change_from_baseline":"residential"
}

for old_field, new_field in field_rename_mapping.items():
    Australia.update_many({}, {"$rename": {old_field: new_field}}) # Looping through the mapping to rename fields



In [20]:
# convert date to datetime datatype

Australia=coviddb['Australia']

# change the datatype of date to datetime
old_date = "date"

# Update documents to change the datatype of the date field to datetime
Australia.update_many({}, [
    {"$set": {"new_date": {"$toDate": "$" + old_date}}},
    {"$unset":"date"}
])


In [22]:
Australia=coviddb['Australia']
Australia.update_many({}, {"$unset": {"country": ""}})


In [23]:
list(Australia.find())

[{'_id': ObjectId('650519654250574b1fd8ee93'),
  'state': '',
  'region': '',
  'retail_and_recreation': 4,
  'grocery_and_pharmacy': 3,
  'parks': -2,
  'transit_stations': 3,
  'workplaces': 3,
  'residential': 0,
  'new_date': datetime.datetime(2020, 2, 15, 0, 0)},
 {'_id': ObjectId('650519654250574b1fd8ee94'),
  'state': '',
  'region': '',
  'retail_and_recreation': 3,
  'grocery_and_pharmacy': 5,
  'parks': 9,
  'transit_stations': 3,
  'workplaces': -1,
  'residential': 0,
  'new_date': datetime.datetime(2020, 2, 16, 0, 0)},
 {'_id': ObjectId('650519654250574b1fd8ee95'),
  'state': '',
  'region': '',
  'retail_and_recreation': -1,
  'grocery_and_pharmacy': 0,
  'parks': -6,
  'transit_stations': 7,
  'workplaces': 17,
  'residential': -2,
  'new_date': datetime.datetime(2020, 2, 17, 0, 0)},
 {'_id': ObjectId('650519654250574b1fd8ee96'),
  'state': '',
  'region': '',
  'retail_and_recreation': -3,
  'grocery_and_pharmacy': -2,
  'parks': -13,
  'transit_stations': 7,
  'workpla

In [32]:
Aus_df=pd.DataFrame(list(coviddb.Australia.find())) #To have a visual of how the data looks now
Aus_df

,_id,state,region,retail_and_recreation,grocery_and_pharmacy,parks,transit_stations,workplaces,residential,new_date
0,650519654250574b1fd8ee93,,,4,3,-2,3,3,0,2020-02-15
1,650519654250574b1fd8ee94,,,3,5,9,3,-1,0,2020-02-16
2,650519654250574b1fd8ee95,,,-1,0,-6,7,17,-2,2020-02-17
3,650519654250574b1fd8ee96,,,-3,-2,-13,7,14,-1,2020-02-18
4,650519654250574b1fd8ee97,,,-1,-1,-6,8,13,-1,2020-02-19
...,...,...,...,...,...,...,...,...,...,...
496629,650525e19884ca171d66411c,Western Australia,Town of Victoria Park,6,9,-56,-42,-2,,2022-07-17
496630,650525e19884ca171d66411d,Western Australia,Town of Victoria Park,1,12,-17,-20,2,6,2022-07-18
496631,650525e19884ca171d66411e,Western Australia,Town of Victoria Park,8,16,-2,-16,9,9,2022-07-19
496632,650525e19884ca171d66411f,Western Australia,Town of Victoria Park,5,6,-18,-17,5,7,2022-07-20


#### Create collection for the cities in each Australian states



In [26]:
# Create collection that contains only data for Brisbane, Queensland

Australia = coviddb["Australia"]
brs = coviddb["Brisbane"]

filter_criteria = {"state": "Queensland", "region": "Brisbane City"}

aus_data = Australia.find(filter_criteria)

brs.insert_many(aus_data)

brs.update_many({}, {"$unset": {"state": ""}})

In [ ]:
# Create collection that contains only data for Hobart, Tasmania

Australia = coviddb["Australia"]
hob = coviddb["Hobart"]

filter_criteria = {"state": "Tasmania", "region":"City of Hobart"}

aus_data = Australia.find(filter_criteria)

hob.insert_many(aus_data)
hob.update_many({}, {"$unset": {"state": ""}})

In [ ]:
# Create collection that contains only data for Perth, WA

Australia = coviddb["Australia"]
pet = coviddb["Perth"]

filter_criteria = {"state": "Western Australia", "region":"City of Perth"}

aus_data = Australia.find(filter_criteria)

pet.insert_many(aus_data)
pet.update_many({}, {"$unset": {"state": ""}})

In [ ]:
# Create collection that contains only data for Adelaide, Queensland

Australia = coviddb["Australia"]
adl = coviddb["Adelaide"]

filter_criteria = {"state": "South Australia", "region":"Adelaide City Council"}

aus_data = Australia.find(filter_criteria)

adl.insert_many(aus_data)
adl.update_many({}, {"$unset": {"state": ""}})

In [ ]:
# Create collection that contains only data for Sydney, NSW

Australia = coviddb["Australia"]
syd = coviddb["Sydney"]

filter_criteria = {"state": "New South Wales", "region":"City of Sydney"}

aus_data = Australia.find(filter_criteria)

syd.insert_many(aus_data)
syd.update_many({}, {"$unset": {"state": ""}})

In [ ]:
# Create collection that contains only data for ACT

Australia = coviddb["Australia"]
act = coviddb["Australian Capital Territory"]

filter_criteria = {"state": "Australian Capital Territory"}

aus_data = Australia.find(filter_criteria)

act.insert_many(aus_data)
act.update_many({}, {"$unset": {"state": ""}})

In [ ]:
# Create collection that contains only data for Darwin, Northern Territory

Australia = coviddb["Australia"]
dwn = coviddb["Darwin"]

filter_criteria = {"state": "Northern Territory", "region":"City of Darwin"}

aus_data = Australia.find(filter_criteria)

dwn.insert_many(aus_data)
dwn.update_many({}, {"$unset": {"state": ""}})

In [ ]:
# Create collection that contains only data for Melbourne, Victoria

Australia = coviddb["Australia"]
mel = coviddb["Melbourne"]

filter_criteria = {"state": "Victoria", "region":"City of Melbourne"}

aus_data = Australia.find(filter_criteria)

mel.insert_many(aus_data)
mel.update_many({}, {"$unset": {"state": ""}})

In [33]:
# To view all the created collections

list_of_collections = coviddb.list_collection_names()

# Print the list of collection names
for collection_lists in list_of_collections:
    print(collection_lists)

Melbourne
Sydney
Australian Capital Territory
Perth
Hobart
Darwin
Australia
Brisbane
Mobility
Adelaide


In [35]:
import numpy as np
import pandas as pd

## Examine the data for each city

In [47]:
### ACT

act=coviddb['Australian Capital Territory']
act_data = list(act.find({"new_date":
                 {"$gte":datetime.datetime(2020, 1, 1),
                  "$lte":datetime.datetime(2022, 12, 31)}}))
act_df = pd.DataFrame(act_data)
del(act_df['_id'])
act_df.replace('', np.nan,inplace=True)
act_df['region'].fillna('Canberra', inplace=True)
print("Number of missing values:\n", act_df.isna().sum())
act_df.fillna(method='bfill', inplace=True)
act_df


Number of missing values:
 region                    0
retail_and_recreation     0
grocery_and_pharmacy      0
parks                    25
transit_stations          3
workplaces                3
residential               0
new_date                  0
dtype: int64


,region,retail_and_recreation,grocery_and_pharmacy,parks,transit_stations,workplaces,residential,new_date
0,Canberra,7,5,28.0,4.0,6.0,-1,2020-02-15
1,Canberra,8,19,34.0,16.0,2.0,-2,2020-02-16
2,Canberra,-1,10,9.0,21.0,25.0,-4,2020-02-17
3,Canberra,-2,-1,-11.0,12.0,24.0,-2,2020-02-18
4,Canberra,2,9,12.0,17.0,22.0,-3,2020-02-19
...,...,...,...,...,...,...,...,...
883,Canberra,2,11,-15.0,-30.0,0.0,1,2022-07-17
884,Canberra,-11,11,-36.0,-31.0,-9.0,6,2022-07-18
885,Canberra,-9,6,-40.0,-33.0,-6.0,9,2022-07-19
886,Canberra,-9,9,-34.0,-33.0,-9.0,8,2022-07-20


In [38]:
# Brisbane
qld=coviddb['Brisbane']
qld_data = list(qld.find({"new_date":
                 {"$gte":datetime.datetime(2020, 1, 1),
                  "$lte":datetime.datetime(2022, 12, 31)}}))
qld_df = pd.DataFrame(qld_data)
del(qld_df['_id'])
qld_df.replace('', np.nan,inplace=True)
print("Number of missing values:\n", qld_df.isna().sum())
qld_df

Number of missing values:
 region                   0
retail_and_recreation    0
grocery_and_pharmacy     0
parks                    0
transit_stations         0
workplaces               0
residential              0
new_date                 0
dtype: int64


,region,retail_and_recreation,grocery_and_pharmacy,parks,transit_stations,workplaces,residential,new_date
0,Brisbane City,4,6,7,-4,3,0,2020-02-15
1,Brisbane City,4,5,-3,-3,-2,1,2020-02-16
2,Brisbane City,-1,4,-5,4,14,-2,2020-02-17
3,Brisbane City,-1,2,10,4,10,-2,2020-02-18
4,Brisbane City,-2,1,2,5,9,-1,2020-02-19
...,...,...,...,...,...,...,...,...
883,Brisbane City,6,6,-7,-26,-17,2,2022-07-17
884,Brisbane City,-10,2,-28,-31,-13,6,2022-07-18
885,Brisbane City,-10,5,-23,-33,-13,9,2022-07-19
886,Brisbane City,-10,4,-34,-31,-17,8,2022-07-20


In [40]:
# Darwin in Northern Territory Year 2020
dwn=coviddb['Darwin']
dwn_data = list(dwn.find({"new_date":
                 {"$gte":datetime.datetime(2020, 1, 1),
                  "$lte":datetime.datetime(2022, 12, 31)}}))
dwn_df = pd.DataFrame(dwn_data)
del(dwn_df['_id'])
dwn_df.replace('', np.nan,inplace=True)
print("The missing values are:\n", dwn_df.isna().sum())
dwn_df.fillna(method='bfill', inplace=True)
dwn_df

The missing values are:
 region                    0
retail_and_recreation    20
grocery_and_pharmacy     25
parks                    25
transit_stations         25
workplaces                6
residential              19
new_date                  0
dtype: int64


,region,retail_and_recreation,grocery_and_pharmacy,parks,transit_stations,workplaces,residential,new_date
0,City of Darwin,4.0,15.0,13.0,-7.0,-1.0,1.0,2020-02-15
1,City of Darwin,3.0,17.0,13.0,1.0,2.0,1.0,2020-02-16
2,City of Darwin,2.0,16.0,41.0,4.0,21.0,-2.0,2020-02-17
3,City of Darwin,-5.0,3.0,6.0,-1.0,14.0,-1.0,2020-02-18
4,City of Darwin,3.0,14.0,23.0,-5.0,15.0,-1.0,2020-02-19
...,...,...,...,...,...,...,...,...
883,City of Darwin,27.0,17.0,90.0,3.0,-1.0,0.0,2022-07-17
884,City of Darwin,18.0,35.0,111.0,4.0,15.0,0.0,2022-07-18
885,City of Darwin,13.0,25.0,83.0,-5.0,14.0,4.0,2022-07-19
886,City of Darwin,16.0,27.0,76.0,-2.0,18.0,1.0,2022-07-20


In [41]:
# Hobart in Tansmania

# Year 2020
hob=coviddb['Hobart']
hob_data = list(hob.find({"new_date":
                 {"$gte":datetime.datetime(2020, 1, 1),
                  "$lte":datetime.datetime(2020, 12, 31)}}))
hob_df = pd.DataFrame(hob_data)
del(hob_df['_id'])
hob_df.replace('', np.nan,inplace=True)
print("The missing values are:\n", hob_df.isna().sum())
hob_df.fillna(method='bfill', inplace=True)
hob_df

The missing values are:
 region                    0
retail_and_recreation     0
grocery_and_pharmacy     19
parks                    22
transit_stations         22
workplaces               22
residential              89
new_date                  0
dtype: int64


,region,retail_and_recreation,grocery_and_pharmacy,parks,transit_stations,workplaces,residential,new_date
0,City of Hobart,6,5.0,-9.0,7.0,-2.0,-2.0,2020-02-15
1,City of Hobart,-1,8.0,-2.0,11.0,4.0,-2.0,2020-02-16
2,City of Hobart,2,16.0,2.0,25.0,17.0,-2.0,2020-02-17
3,City of Hobart,0,17.0,19.0,20.0,11.0,-1.0,2020-02-18
4,City of Hobart,2,8.0,-13.0,16.0,14.0,-2.0,2020-02-19
...,...,...,...,...,...,...,...,...
313,City of Hobart,-9,-12.0,-31.0,-11.0,-13.0,10.0,2020-12-27
314,City of Hobart,-23,-9.0,8.0,-51.0,-66.0,10.0,2020-12-28
315,City of Hobart,-17,-2.0,4.0,-48.0,-49.0,8.0,2020-12-29
316,City of Hobart,-12,-6.0,9.0,-50.0,-45.0,7.0,2020-12-30


In [42]:
# Sydney in New South Wales
syd=coviddb['Sydney']
syd_data = list(syd.find({"new_date":
                 {"$gte":datetime.datetime(2020, 1, 1),
                  "$lte":datetime.datetime(2022, 12, 31)}}))
syd_df = pd.DataFrame(syd_data)
del(syd_df['_id'])
syd_df.replace('', np.nan,inplace=True)
print("The missing values are:\n", syd_df.isna().sum())
syd_df

The missing values are:
 region                   0
retail_and_recreation    0
grocery_and_pharmacy     0
parks                    0
transit_stations         0
workplaces               0
residential              0
new_date                 0
dtype: int64


,region,retail_and_recreation,grocery_and_pharmacy,parks,transit_stations,workplaces,residential,new_date
0,City of Sydney,6,9,25,14,2,-1,2020-02-15
1,City of Sydney,2,13,57,8,0,-1,2020-02-16
2,City of Sydney,-1,6,6,6,9,-1,2020-02-17
3,City of Sydney,3,4,-1,5,7,-1,2020-02-18
4,City of Sydney,2,3,11,7,4,-1,2020-02-19
...,...,...,...,...,...,...,...,...
883,City of Sydney,-19,-19,39,-12,-21,2,2022-07-17
884,City of Sydney,-47,-37,-24,-43,-40,13,2022-07-18
885,City of Sydney,-43,-38,-46,-40,-32,16,2022-07-19
886,City of Sydney,-42,-38,-37,-39,-35,13,2022-07-20


In [43]:
# Melbourne in Victoria

mel=coviddb['Melbourne']
mel_data = list(mel.find({"new_date":
                 {"$gte":datetime.datetime(2020, 1, 1),
                  "$lte":datetime.datetime(2022, 12, 31)}}))
mel_df = pd.DataFrame(mel_data)
del(mel_df['_id'])
mel_df.replace('', np.nan,inplace=True)
print("The missing values are:\n", mel_df.isna().sum())
mel_df

The missing values are:
 region                   0
retail_and_recreation    0
grocery_and_pharmacy     0
parks                    0
transit_stations         0
workplaces               0
residential              0
new_date                 0
dtype: int64


,region,retail_and_recreation,grocery_and_pharmacy,parks,transit_stations,workplaces,residential,new_date
0,City of Melbourne,1,-1,-31,6,0,2,2020-02-15
1,City of Melbourne,3,7,-8,2,-2,0,2020-02-16
2,City of Melbourne,3,9,-1,10,7,-2,2020-02-17
3,City of Melbourne,0,-2,-23,4,6,1,2020-02-18
4,City of Melbourne,8,4,-19,5,4,1,2020-02-19
...,...,...,...,...,...,...,...,...
883,City of Melbourne,-15,-11,-51,-27,-2,3,2022-07-17
884,City of Melbourne,-43,-28,-62,-52,-31,13,2022-07-18
885,City of Melbourne,-33,-21,-48,-43,-24,14,2022-07-19
886,City of Melbourne,-38,-18,-57,-45,-27,13,2022-07-20


In [45]:
# Adelaide in South Australia
adl=coviddb['Adelaide']
adl_data = list(adl.find({"new_date":
                 {"$gte":datetime.datetime(2020, 1, 1),
                  "$lte":datetime.datetime(2022, 12, 31)}}))
adl_df = pd.DataFrame(adl_data)
del(adl_df['_id'])
adl_df.replace('', np.nan,inplace=True)
print("The missing values are:\n",adl_df.isna().sum())
adl_df.fillna(method='bfill', inplace=True)
adl_df

The missing values are:
 region                     0
retail_and_recreation      0
grocery_and_pharmacy      25
parks                     25
transit_stations           0
workplaces                58
residential              426
new_date                   0
dtype: int64


,region,retail_and_recreation,grocery_and_pharmacy,parks,transit_stations,workplaces,residential,new_date
0,Adelaide City Council,32,5.0,165.0,26,10.0,9.0,2020-02-15
1,Adelaide City Council,-1,12.0,84.0,-11,10.0,9.0,2020-02-16
2,Adelaide City Council,5,9.0,24.0,-1,10.0,9.0,2020-02-17
3,Adelaide City Council,12,7.0,83.0,3,3.0,9.0,2020-02-18
4,Adelaide City Council,9,7.0,81.0,1,5.0,9.0,2020-02-19
...,...,...,...,...,...,...,...,...
883,Adelaide City Council,-31,17.0,-45.0,-42,10.0,8.0,2022-07-17
884,Adelaide City Council,-30,-10.0,-34.0,-28,-2.0,8.0,2022-07-18
885,Adelaide City Council,-23,-6.0,-24.0,-25,-6.0,11.0,2022-07-19
886,Adelaide City Council,-25,-4.0,-30.0,-27,-1.0,9.0,2022-07-20


In [46]:
# Perth in Australia

pet=coviddb['Perth']
pet_data = list(pet.find({"new_date":
                 {"$gte":datetime.datetime(2020, 1, 1),
                  "$lte":datetime.datetime(2022, 12, 31)}}))
pet_df = pd.DataFrame(pet_data)
del(pet_df['_id'])
pet_df.replace('', np.nan,inplace=True)
print("Number of missing values:\n", pet_df.isna().sum())
pet_df.fillna(method='bfill', inplace=True)
pet_df


Number of missing values:
 region                     0
retail_and_recreation      0
grocery_and_pharmacy      25
parks                      0
transit_stations           0
workplaces                45
residential              342
new_date                   0
dtype: int64


,region,retail_and_recreation,grocery_and_pharmacy,parks,transit_stations,workplaces,residential,new_date
0,City of Perth,23,5.0,7,41,8.0,0.0,2020-02-15
1,City of Perth,13,5.0,17,14,8.0,0.0,2020-02-16
2,City of Perth,-1,5.0,-1,15,8.0,0.0,2020-02-17
3,City of Perth,-1,4.0,-5,13,7.0,0.0,2020-02-18
4,City of Perth,1,5.0,1,7,9.0,0.0,2020-02-19
...,...,...,...,...,...,...,...,...
883,City of Perth,-23,-17.0,-55,-23,3.0,8.0,2022-07-17
884,City of Perth,-34,-15.0,-39,-10,-11.0,8.0,2022-07-18
885,City of Perth,-30,-11.0,-38,-9,-10.0,11.0,2022-07-19
886,City of Perth,-33,-10.0,-45,-11,-6.0,8.0,2022-07-20
